In [227]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import joblib
import duckdb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity
import umap
import nltk
import re
from sentence_transformers import SentenceTransformer

# Import from local modules
import sys
sys.path.append('../src')
import clustering_analysis
import importlib
importlib.reload(clustering_analysis)

from cleaning import minimal_clean
from clustering_analysis import cluster_with_umap_hdbscan, recluster_noise, print_cluster_examples, summarize_clusters, metacluster_preview

# Download necessary NLTK resources
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

# Path for 1 million English sample questions
load_dotenv()
unlabeled_path = os.getenv("DATA_UNLABELED")
unlabeled_embeddings_path = os.getenv("DATA_UNLABELED_EMBEDDINGS")
data_dir = os.getenv("DATA_DIR")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ruggb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ruggb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Unlabeled Questions
Separate notebook dedicated to clustering questions without a pre-existing 'question_topic' label

In [197]:
# load data
questions = pd.read_parquet(unlabeled_path)

questions = questions.drop(['response_user_type', 'response_user_status',
       'response_user_country_code', 'response_user_gender',
       'response_user_dob', 'response_user_created_at', 'response_id',
       'response_user_id', 'response_language', 'response_content',
       'response_topic', 'response_sent'], axis=1)

questions.shape

(643148, 12)

In [198]:
# Apply separate cleaning function that just normalizes spacing and removes very common "Q" boilerplate
questions['Q_basic_clean'] = questions['question_content'].apply(minimal_clean)

In [ ]:
# Check if embeddings file exists before generating

if os.path.exists(unlabeled_embeddings_path):
    print("Embedding file exists — loading from disk.")
    data = np.load(unlabeled_embeddings_path, allow_pickle=True)
    embeddings = data["embeddings"]
    question_ids = data["question_ids"]  # shouldn't be necessary, but saved just in case of a mix-up

else:
    print("Embedding file not found — generating embeddings.")
    model = SentenceTransformer('all-MiniLM-L6-v2')

    texts = questions['Q_basic_clean'].tolist()
    question_ids = questions['question_id'].to_numpy()

    embeddings = model.encode(
        texts,
        batch_size=64,
        show_progress_bar=True
    )
# Save embeddings to disk along with question IDs in case of mixup
    np.savez_compressed(unlabeled_embeddings_path, embeddings=embeddings, question_ids=question_ids)
    print("Saved embeddings to disk.")

questions['embedding'] = list(embeddings)

Embedding file exists — loading from disk.


In [ ]:
## Perform UMAP + HDBSCAN clustering
# Commented out to avoid re-running unnecessarily or acccidentally overwriting

# unlabeled_df, unlabeled_umap, unlabeled_clusterer = cluster_with_umap_hdbscan(
#     df=questions,
#     sample_size=100_000,
#     umap_params={
#         "n_neighbors": 30,
#         "n_components": 5,
#         "metric": "cosine",
#         "random_state": 42
#     },
#     hdbscan_params={
#         "min_cluster_size": 1500,
#         "min_samples": 15,
#         "metric": "euclidean"
#     }
# )

Finished clustering in 4285.2 seconds
Noise ratio: 43.21%
Clusters found: 81
Approx. silhouette score (excluding noise): 0.625


In [ ]:
## Saves result of large clustering to disk if not already present
# files_to_save = {
#     "unlabeled_hdbscan_model.pkl": unlabeled_clusterer,
#     "unlabeled_umap_embedding.pkl": unlabeled_umap,
#     "unlabeled_clustered_df.parquet": unlabeled_df
# }

# for filename, obj in files_to_save.items():
#     path = os.path.join(data_dir, filename)
#     if not os.path.exists(path):
#         if filename.endswith(".parquet"):
#             obj.to_parquet(path)
#         else:
#             joblib.dump(obj, path)
#         print(f"Saved {filename}")
#     else:
#         print(f"{filename} already exists. Skipping save.")



unlabeled_hdbscan_model.pkl already exists. Skipping save.
unlabeled_umap_embedding.pkl already exists. Skipping save.
unlabeled_clustered_df.parquet already exists. Skipping save.


In [ ]:
## Runs HDBSCAN again on the noise cluster to extract more clusters
# Adjust paramaters for smaller subset of data
hdbscan_params_noise = {
    "min_cluster_size": 400,
    "min_samples": 10,
    "metric": "euclidean"
}

noise_labels_shifted, noise_clusterer = recluster_noise(
    umap_embeddings=unlabeled_umap,
    labels=unlabeled_df["cluster"].to_numpy(),
    hdbscan_params=hdbscan_params_noise
)

Reclustered noise points: 277924
New noise ratio: 56.82%
Clusters found in noise: 136


In [ ]:
## Preview new clusters obtained from noise cluster for quick quality check

# Select only the original noise rows
noise_mask = unlabeled_df['cluster'] == -1
noise_df = unlabeled_df.loc[noise_mask, ['Q_basic_clean']].copy()

# Assign the new noise cluster labels (length matches noise_df)
noise_df['new_cluster'] = noise_labels_shifted

print_cluster_examples(noise_df, text_column='Q_basic_clean', cluster_column='new_cluster',examples_per_cluster=5)



--- Cluster 187 (size=10516) ---
- what are tha advantages of dairy farming and tha considerations set during tha setup of a dairy farm?
- What Leads Farmers Failure To Calculate Profits And Losses At The End Of The Season.
- If Am Poor I Want To Bcome A Farmer What Can I Do
- What are the factors to consider when setting up farm building?
- Many people dislike farming Why is it so?

--- Cluster 110 (size=5219) ---
- WHAT IS SOIL TEXTURE?
- HOW CAN YOU CONTROL SOILEROSION
- WHEN SHEET EROSION OCCUR ON YOUR FARM WHAT WILL YOU DO?
- ,which soil doesnt require too much water
- ,do soil affect water retention

--- Cluster 166 (size=2877) ---
- Acaricides Used In Controlling Oxalis
- where can I buy Victoria sate herbicide in mbarara
- How iz it harmful weed'g us'g chemicals.
- Why are some farmers use weedmaster to kill weeds?
- WHEN I SPRAY WEEDS WITH CHEMICALS DO THEY TAKE A LONG TIME WITH OUT COMING BACK?

--- Cluster 182 (size=1860) ---
- ,whech taip of beenci is the best grow in kita

In [ ]:
# Integrates new noise cluster labels back into the main dataframe
unlabeled_df.loc[unlabeled_df["cluster"] == -1, "cluster"] = noise_labels_shifted


In [223]:
unlabeled_summary = summarize_clusters(unlabeled_df, 
                        text_col='Q_basic_clean', 
                        cluster_col='cluster', 
                        top_n_words=5, 
                        meta_col='meta_label', 
                        sample_questions=5, 
                        random_samples=True, 
                        preview=True)


=== Cluster -1 (size=157906) ===
Keywords: use, wefarm, does, best, wat
Meta: nan
Sample questions:
  - can i get an electrician post in your organisation?
  - ,Are they capable to survive or
  - Where can i get mint seeds pliz members help
  - State the function of acotyledon.
  - SOO FOR WHAT

=== Cluster 0 (size=3449) ===
Keywords: mulching, mulch, importance, mulches, garden
Meta: 4.0
Sample questions:
  - Why Do People Mulch Their Garden?
  - Good Simon, Which Materials Do you use for mulching inorder keep the moist in the garden?
  - what is mulching?
  - am ask a qn to d'stance of mulching from bananaplant to other afarmers for more lnf?
  - What Is Mulching ?

=== Cluster 1 (size=4401) ===
Keywords: ticks, tick, control, best, farm
Meta: 2.0
Sample questions:
  - Please Told Me Medicine Of Poison Of Tick
  - what are the harmful effects of the tick
  - Which is the best caracide for killing ticks?
  - Which are the 2 example of two host tick
  - mention three types of tick fou

In [ ]:
# Define metacluster titles
meta_titles = {
    -1: "Uncategorized", 
    1: "Soil, Fertilizer, & Animal Feed", 
    2: "Pests & Disease", 
    3: "Animal Husbandry", 
    4: "Planting & Growth", 
    5: "Markets", 
    6: "Finance & Loans", 
    7: "Farming Equipment & Materials", 
    8: "Weather & Environment", 
    9: "Wefarm Platform",
    10: "Personal Communication"
}


In [ ]:
# Map clusters to metaclusters based on ChatGPT feedback. 
# Input the prompt found in /notes/LLM_prompts.txt followed by the output of summarize_clusters(), keeping within character limit (I submitted chunks of 40-50 clusters at a time)

cluster_to_meta = {
    **dict.fromkeys([0, 12, 15, 16, 52, 53, 54, 57, 58, 59, 60, 83, 84, 87, 88, 91, 92, 98, 101, 108, 113, 118, 124, 126, 130, 137, 147, 151, 173, 175, 177, 182, 183, 188], 4),  # Planting & Growth
    **dict.fromkeys([1, 7, 9, 13, 34, 35, 45, 46, 47, 48, 85, 99, 107, 131, 133, 140, 144, 150, 165, 166, 168, 169], 2),  # Pests & Disease
    **dict.fromkeys([2, 3, 4, 5, 6, 10, 11, 14, 24, 26, 27, 29, 32, 55, 56, 81, 90, 95, 96, 102, 103, 117, 134, 146, 156, 162, 178, 179, 191, 203], 3),  # Animal Husbandry
    **dict.fromkeys([8, 19, 20, 21, 22, 23, 33, 39, 40, 41, 43, 44, 49, 50, 65, 69, 75, 82, 94, 106, 110, 111, 114, 115, 116, 119, 120, 121, 123, 141, 142, 149, 153, 154, 155, 159, 164, 181, 210], 1),  # Soil, Fertilizer, & Animal Feed
    **dict.fromkeys([74, 122, 132, 148, 184, 185, 204, 205, 206, 207, 208, 209, 176, 174], 5),  # Markets
    **dict.fromkeys([17, 18, 71, 72, 186, 167, 163], 6),  # Finance & Loans
    **dict.fromkeys([25, 28, 31, 42, 76, 86, 93, 100, 112, 129], 7),  # Farming Equipment & Materials
    **dict.fromkeys([30, 51, 97, 109, 128, 138, 180, 192], 8),  # Weather & Environment
    **dict.fromkeys([37, 61, 62, 67, 77, 80, 89, 139, 187, 196, 198, 199], 9),  # Wefarm Platform
    **dict.fromkeys([36, 38, 63, 64, 68, 70, 73, 78, 79, 125, 135, 136, 152, 157, 158, 160, 161, 189, 190, 193, 194, 195, 197, 200, 201, 202, 211, 212, 213, 214, 215, 216], 10),  # Personal Communication
    **dict.fromkeys([66, 104, 105, 127, 170], -1)  # Uncategorized
}
unlabeled_df['meta_label'] = unlabeled_df['cluster'].map(cluster_to_meta)


In [235]:
meta_titles = {
    -1: "Uncategorized", 
    1: "Soil & Fertilizer", 
    2: "Pests & Disease", 
    3: "Animal Husbandry", 
    4: "Planting & Growth", 
    5: "Markets", 
    6: "Finance & Loans", 
    7: "Farming Equipment & Materials", 
    8: "Weather & Environment", 
    9: "Wefarm Platform",
    10: "Personal Communication"
}
unlabeled_df['meta_label_title'] = unlabeled_df['meta_label'].map(meta_titles)
unlabeled_df.loc[unlabeled_df['cluster'] == -1, 'meta_label'] = -1

In [236]:
unlabeled_df['meta_label_title'].value_counts()

meta_label_title
Uncategorized                    162234
Animal Husbandry                  89566
Soil & Fertilizer                 89456
Personal Communication            72921
Planting & Growth                 61526
Pests & Disease                   50452
Wefarm Platform                   41485
Markets                           24591
Farming Equipment & Materials     21266
Weather & Environment             14615
Finance & Loans                   11772
Name: count, dtype: int64

In [243]:
metacluster_preview(unlabeled_summary, 
                    metacluster_num=10, 
                    meta_titles=meta_titles)

Previewing Meta-cluster 10: Personal Communication

=== Cluster 36 (size=4860) ===
Keywords: sex, vagina, penis, semen, wat
Sample questions:
  - bacuse it sweet
  - How to be fucked
  - Things To Do To Make My Penis Erect At Night During Sex
  - WHERE CAN I GET BEST SEMEN FOR BUNGOMA COUNTY
  - Is there also women

=== Cluster 38 (size=3749) ===
Keywords: win, winner, winners, champion, won
Sample questions:
  - what do you consider for someone to be a champion
  - what prize would you gve me
  - WHO WIN yesterday
  - UESTION REPLY WIN SH 10000
  - please i want my prize

=== Cluster 63 (size=1536) ===
Keywords: located, place, stay, exactly, location
Sample questions:
  - Where a you situated
  - i meant outside the country
  - E Where are you now?
  - WHERE CAN I FIND YOU
  - where do u leave?

=== Cluster 64 (size=4410) ===
Keywords: join, group, job, want, member
Sample questions:
  - how can l become a member
  - n what are your names all members
  - where can i get someone want 

In [244]:
unlabeled_df

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,question_user_type,question_user_status,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,Q_basic_clean,embedding,cluster,meta_label,meta_label_title
0,4107267,319019,eng,Q Living Kasese Asks Medincine For (ekifuruto),None,2017-12-08 19:23:23+00,farmer,live,ug,None,None,2017-08-30 11:39:28+00,Living Kasese Asks Medincine For (ekifuruto),"[-0.017760053, 0.027590225, -0.029300023, -0.0...",79,10.0,Personal Communication
1,4107349,405490,eng,Q: Hw A Person Can Improve Milk Production In...,None,2017-12-08 19:41:49+00,farmer,zombie,ug,None,None,2017-09-29 17:36:14+00,Hw A Person Can Improve Milk Production In His...,"[0.007432304, -0.037007444, -0.017815048, 0.00...",5,3.0,Animal Husbandry
2,4107354,451635,eng,What are the effect of using alot of chemicals...,None,2017-12-08 19:43:36+00,farmer,destroyed,ke,None,None,2017-10-19 17:17:19+00,What are the effect of using alot of chemicals...,"[0.09056131, 0.043528926, 0.08471735, -0.04495...",141,1.0,Soil & Fertilizer
3,4107424,571641,eng,Q : How many times is the land likely to be p...,None,2017-12-08 20:06:41+00,farmer,destroyed,ke,None,None,2017-12-07 17:34:20+00,: How many times is the land likely to be prep...,"[0.053764395, 0.024894716, 0.0038809194, 0.027...",76,7.0,Farming Equipment & Materials
4,4107802,574940,eng,Q answer quickly please!,None,2017-12-09 04:07:43+00,farmer,zombie,ug,None,None,2017-12-08 18:33:04+00,answer quickly please!,"[0.024155334, 0.09032307, -0.01413228, 0.06118...",73,10.0,Personal Communication
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643143,59086651,3436194,eng,How cost of harsh fruit per kilo reply,None,2021-10-13 18:46:37.57081+00,farmer,live,ke,None,None,2020-11-12 17:49:23.116921+00,How cost of harsh fruit per kilo reply,"[0.05027966, 0.041477513, 0.023335036, 0.07118...",52,4.0,Planting & Growth
643144,59086652,3551028,eng,How can one maintain soil fertility in the farm?,None,2021-10-13 18:47:19.248937+00,farmer,live,ke,None,None,2021-01-27 17:18:47.958894+00,How can one maintain soil fertility in the farm?,"[-0.019021185, 0.033774376, -0.020837162, 0.01...",22,1.0,Soil & Fertilizer
643145,59086816,1914124,eng,Q Where Do i Sale Milk In Kisumu,None,2021-10-14 04:29:40.732573+00,farmer,live,ke,None,None,2019-02-25 11:08:34.548744+00,Where Do i Sale Milk In Kisumu,"[0.07002282, -0.032133285, -0.025083825, 0.016...",5,3.0,Animal Husbandry
643146,59086934,2896498,eng,Q what is the name of a females be,None,2021-10-14 07:08:23.534742+00,farmer,live,ke,None,None,2020-02-18 17:43:04.434573+00,what is the name of a females be,"[0.009304676, -0.016631693, -0.010773809, 0.04...",36,10.0,Personal Communication


In [ ]:
umap_2d = umap.UMAP(
    n_components=2,
    n_neighbors=50,
    min_dist=0.1,
    metric="cosine",
    random_state=42
)

embedding_2d = umap_2d.fit_transform(unlabeled_embeddings)


### Automated meta-clustering attempt
(not successful)

In [112]:
from umap import UMAP
from sklearn.cluster import AgglomerativeClustering
import numpy as np

# Compute centroids
cluster_centroids = (
    unlabeled_df[unlabeled_df['cluster'] != -1]['embedding']
    .groupby(unlabeled_df['cluster'])
    .apply(lambda x: np.mean(np.vstack(x), axis=0))
)
centroids = np.vstack(cluster_centroids.values)

# Optional: reduce dimensionality before meta-clustering
umap_model = UMAP(n_components=2, metric='cosine', random_state=42)
centroids_2d = umap_model.fit_transform(centroids)

# Agglomerative clustering on reduced embeddings
meta_clusterer = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.55,
    linkage='average'
)
meta_labels_no_noise = meta_clusterer.fit_predict(centroids_2d)

# Map back to original clusters
meta_labels = dict(zip(cluster_centroids.index, meta_labels_no_noise))
meta_labels[-1] = -1
unlabeled_df['meta_cluster'] = unlabeled_df['cluster'].map(meta_labels)




In [113]:
for lbl, group in unlabeled_df.groupby("meta_cluster"):
    print("Number of Meta-clusters:", len(unlabeled_df['meta_cluster'].unique()))
    print(f"\n--- Meta-cluster {lbl} ---")
    print(group["cluster"].value_counts())

Number of Meta-clusters: 29

--- Meta-cluster -1 ---
cluster
-1    22134
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 0 ---
cluster
9     2667
13    1530
10     866
8      668
41     449
42     169
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 1 ---
cluster
3     904
1     668
64    350
65    299
47    254
49    170
48    150
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 2 ---
cluster
39    243
50    225
77    224
45    207
57    193
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 3 ---
cluster
30    877
68    356
63    344
62    338
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 4 ---
cluster
36    1995
34    1339
92     409
94     360
90     218
84     193
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 5 ---
cluster
6     1687
18    1156
51     218
52     183
Name: count, dtype: int64
Number of Meta-clusters: 29

--- Meta-cluster 6 -

In [114]:
for lbl, group in unlabeled_df.groupby("meta_cluster"):
    print(f"\n--- Meta-cluster {lbl} ---")
    print(group["Q_basic_clean"].sample(10).tolist())


--- Meta-cluster -1 ---
['what is tilling of the land', 'Wat are pleateaus ?', 'what is omnivorous', 'Aminah asks: How can i plan matooke? Reply Q25 followed by your response.', 'which Pesticides is the best for controling weevils', 'what is the meaning of the word brids', 'I dot knowe where do we get bt even me i dot where do you get', 'Is God real?', 'What can ido? Am apoutry farmer.', 'What Is The Causal Organism Of Anthrax']

--- Meta-cluster 0 ---
['what is Soil leaching?', 'What is the BEST method of irrigation?', 'How Can I Now The Characteristics Of A Good Soil', 'What Is Soil Leaching?', 'How can soil gain its fertility?', 'discribe for me the proces of cleaning water in the farm', 'How often should I give it water', 'Where will I get a money maker water pumb machine and at how much, in kericho?', 'how the soil loose fertility in the soil', 'give 2 ways how soil is formed']

--- Meta-cluster 1 ---
['which medicine are suitable 2 attack armyworm?', "E if some begins a rat farm